In [29]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
sys.path.append("../../Code")
pd.options.display.max_columns = 999
import files as f
import settings as s
import preprocessing as pre
import sql
import utils
import string
import countries as ctry

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Guests

## Read in the Guest data

In [30]:
guests = f.readin_guests()

Retrieving CSV files from  ../../Data/raw/guests
Processing  ../../Data/raw/guests/out41.csv
Processing  ../../Data/raw/guests/out40.csv
Processing  ../../Data/raw/guests/out42.csv
Processing  ../../Data/raw/guests/out8.csv
Processing  ../../Data/raw/guests/out9.csv
Processing  ../../Data/raw/guests/out43.csv
Processing  ../../Data/raw/guests/out47.csv
Processing  ../../Data/raw/guests/out46.csv
Processing  ../../Data/raw/guests/out44.csv
Processing  ../../Data/raw/guests/out45.csv
Processing  ../../Data/raw/guests/out36.csv
Processing  ../../Data/raw/guests/out22.csv
Processing  ../../Data/raw/guests/out23.csv
Processing  ../../Data/raw/guests/out37.csv
Processing  ../../Data/raw/guests/out21.csv
Processing  ../../Data/raw/guests/out35.csv
Processing  ../../Data/raw/guests/out34.csv
Processing  ../../Data/raw/guests/out20.csv
Processing  ../../Data/raw/guests/out24.csv
Processing  ../../Data/raw/guests/out30.csv
Processing  ../../Data/raw/guests/out18.csv
Processing  ../../Data/raw/gu

## Clean up the data 

In [31]:
guests = pre.clean_guests(guests)

Cleaning Guest Data
Initial number of Guest records: 313199
Trimming white spaces from column names
Cleaning up wishlist column
Removing records with NULL ids: 27388  records
Updated number of guest records: 285811
Drop duplicate records
Updated number of guest records: 225729
Cleaning up quotations
Changing guideNumber to integer
Removing records where guests have no membershipDate 225  records
Updated number of guest records: 225504
Split membershipDate into membershipMonth and membershipYear
Removing records with non-numeric ids:
Updated number of guest records: 225492


## Translations

In [32]:
guests = pre.translate_guests(guests)

Translating months
Distinct number of verifications:  20
Translating verifications
Number of new translations needed for verifications:  0
Distinct number of cities:  27515
Translating cities
Number of new translations needed for cities:  0
Distinct number of cities:  27515
Translating cities
Number of new translations needed for cities:  0


## Extract Country Code

In [33]:
countries = ctry.get_country_info()

In [34]:
c_dic = ctry.add_additional_c_matches(ctry.get_country_dic(countries))

In [35]:
c_dic_invert = utils.invert_dict(c_dic)

In [36]:
guests = ctry.extract_ccode(guests, 'city', c_dic, c_dic_invert)

# Listings

In [37]:
raw_listings = f.readin_file(s.RAW_LISTINGS_DIR)

Retrieving CSV file from  ../../Data/raw/listings.csv


### Listing Information

In [38]:
listings = pre.get_listing_info(raw_listings)

Retrieving 58 columns for listings
Number of NYC listings: 47542


### Host Information

In [39]:
hosts = pre.get_host_info(raw_listings)

Retrieving 15 columns for hosts
Number of NYC hosts: 39452
Removing 'host_' from column names


# Reviews

In [49]:
reviews = f.readin_file(s.RAW_REVIEWS_DIR)

Retrieving CSV file from  ../../Data/raw/reviews.csv


In [50]:
reviews = pre.get_review_info(reviews, listings)

Retrieving 5 columns for reviews
Number of NYC reviews: 952722
Retrieving hostID from listing info


In [51]:
reviews = pre.clean_reviews(reviews)

Cleaning Review Data
Initial number of Review records: 952722
Trimming white spaces 
Cleaning up Line Breaks
Adding cancellation marker


# Host's Trips

In [52]:
host_hosts = f.readin_dir(s.RAW_HOST_HOSTS_DIR, delimiter="==")

Retrieving CSV files from  ../../Data/raw/hostshosts
Processing  ../../Data/raw/hostshosts/hostInfo4.csv
Processing  ../../Data/raw/hostshosts/hostInfo1.csv
Processing  ../../Data/raw/hostshosts/hostInfo2.csv
Processing  ../../Data/raw/hostshosts/hostInfo3.csv


In [53]:
visitation_table = pre.get_host_visitation_table(host_hosts)

In [54]:
visitation_table['visited'] = pre.translate_cities(visitation_table, "visited")

Distinct number of cities:  2496
Translating cities
Number of new translations needed for cities:  0


In [55]:
countries.to_csv("country_info.csv")

In [56]:
visitation_table['country'], visitation_table['ccode'], visitation_table['city'], visitation_table['state'] = ctry.get_city_info(visitation_table["visited"], c_dic)

# Host_Guest_reviews

In [57]:
host_reviews = f.readin_dir(s.RAW_HOST_REVIEWS_DIR, delimiter="==")

Retrieving CSV files from  ../../Data/raw/hostguestreviews
Processing  ../../Data/raw/hostguestreviews/revout8.csv
Processing  ../../Data/raw/hostguestreviews/revout9.csv
Processing  ../../Data/raw/hostguestreviews/revout10.csv
Processing  ../../Data/raw/hostguestreviews/revout11.csv
Processing  ../../Data/raw/hostguestreviews/revout2.csv
Processing  ../../Data/raw/hostguestreviews/revout3.csv
Processing  ../../Data/raw/hostguestreviews/revout1.csv
Processing  ../../Data/raw/hostguestreviews/revout4.csv
Processing  ../../Data/raw/hostguestreviews/revout5.csv
Processing  ../../Data/raw/hostguestreviews/revout7.csv
Processing  ../../Data/raw/hostguestreviews/revout6.csv


In [58]:
host_reviews = pre.rename_host_review_cols(host_reviews)

In [63]:
host_reviews = pre.clean_host_reviews(host_reviews)

Cleaning Review Data
Initial number of Review records: 502812
Trimming white spaces from column names
Trimming white spaces 
Cleaning up Line Breaks
Removing records with NULL guest ids: 0  records
Updated number of host review records: 502802
Cleaning up Total Review column
Updated number of host review records: 502802
Removing records with NULL host ids: 0  records
Updated number of host review records: 502802
Drop duplicate records
Updated number of host review records: 501931
Removing records with non-numeric guest ids:
Updated number of host review records: 501929


# Save to DataBase(Mysql)

In [64]:
sql.create_database()
sql.create_tables()

Database already exists.
Creating table listings: already exists. Recreating table
Creating table listings: OK
Creating table hosts: already exists. Recreating table
Creating table hosts: OK
Creating table guests: already exists. Recreating table
Creating table guests: OK
Creating table hostTrips: already exists. Recreating table
Creating table hostTrips: OK
Creating table guestReviews: already exists. Recreating table
Creating table guestReviews: OK
Creating table hostReviews: already exists. Recreating table
Creating table hostReviews: OK


In [65]:
tables = [guests, hosts, listings, reviews, visitation_table, host_reviews]
table_names = ['guests', 'hosts', 'listings', 'guestReviews', 'hostTrips', 'hostReviews']
sql.push_tables_to_db(tables, table_names)

Pushing 'guests' records
Pushing 'hosts' records
Pushing 'listings' records
Pushing 'guestReviews' records
Pushing 'hostTrips' records
Pushing 'hostReviews' records


# Extras

In [49]:
def add_x(town, state, guess, countries):
    guess = guess.strip()
    if guess in countries.keys():
        return (town, state, countries[guess], guess)
    else:
        s = guess.split(' ')
        if s[-1] in countries.keys():
            return (town, state, countries[s[-1]], s[-1])
        else:
            return (town, state, "UNK", guess)
        

In [673]:
def format_guest_city(tbl, countries):
    new_tbl = pd.concat([tbl, pd.DataFrame(columns=['town-city', 'state', 'ccode', 'country' ])])
    new_tbl['city'] = new_tbl['city'].map(lambda x: x.split(',') if x != None else x)
    for _, r in new_tbl.iterrows():
        c = r['city']
        if c == None or len(c) == 0:
            r[['town-city', 'state', 'ccode', 'country']] = (None, None, None, None)
        elif len(c) == 1:
            if c[0] in countries.keys():
                r[['town-city', 'state', 'ccode', 'country']] = (None, None, countries[c[0]], c[0])
            else:
                sp = c[0].split(' ')
                if len(sp) > 1 and sp[1] in countries.keys():
                    r[['town-city', 'state', 'ccode', 'country']] = (sp[0], None, countries[sp[1]], sp[1])
                else:
                    r[['town-city', 'state', 'ccode', 'country']] = (None, None, "UNK", c)
        elif len(c) == 2:
            if c[-1] in countries.keys():
                r[['town-city', 'state', 'ccode', 'country']] = (c[0], None, countries[c[1].strip()], c[1].strip())
            else:
                sp = c[-1].split(' ')
                if len(sp) > 1 and sp[-1] in countries.keys():
                    r[['town-city', 'state', 'ccode', 'country']] = (sp[0], None, countries[sp[-1]], sp[-1])
                else:
                    r[['town-city', 'state', 'ccode', 'country']] = (c[0], None, "UNK", c[-1].strip())
        else:
            if c[-1] in countries.keys():
                r[['town-city', 'state', 'ccode', 'country']] = (c[0], c[1].strip(), countries[c[-1].strip()], c[-1].strip())
            elif c[-1] in s.states:
                r[['town-city', 'state', 'ccode', 'country']] = (c[0], c[-1].strip(), "US", "United States")
            else:
                r[['town-city', 'state', 'ccode', 'country']] = (c[0], c[1].strip(), "UNK" , c[-1].strip())
    return new_tbl